In [ ]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
from encode import load_biomedclip, simulate_pmc6m_model, encode_image_batch, load_sample_images
from mmd import permutation_test, mmd_rbf, interpret_mmd_result
from tsne_plot import plot_tsne_comparison, plot_mmd_null_distribution, create_publication_figure

print("🧬 Starting OPEN-PMC-18M vs PMC-6M comparison...")


In [ ]:
# Load BiomedCLIP (OPEN-PMC-18M)
model_a, preprocess_a = load_biomedclip()

# Simulate PMC-6M baseline (with quality degradation)
model_b = simulate_pmc6m_model(model_a, noise_level=0.1)

print("✅ Models loaded successfully!")


In [ ]:
# Run MMD permutation test
mmd_observed, p_value, null_mmds = permutation_test(
    embeddings_a, 
    embeddings_b, 
    n_permutations=100,
    mmd_func=mmd_rbf,
    gamma=1.0  # RBF kernel bandwidth
)

# Print detailed results
result_text = interpret_mmd_result(mmd_observed, p_value)
print(result_text)
